In [1]:
import polars as pl
import seaborn as sns
import numpy as np
from scipy import stats

from rna_model import load_training_dataset,get_reactivity_error_cols,get_reactivity_cols
lf = load_training_dataset()

In [2]:
def percent_of_zeros(original_count,non_zero_count):
    """
    The thinking is that if the distribution of zeros is more than 50%, consequently, the MAD will be
    0.0
    """

    # p is the percent of non-zero. The ideal is above 50%
    p = (non_zero_count*100)/original_count
    print(f"{p}%")

reactivity_error_cols = get_reactivity_error_cols()
reactivity_cols = get_reactivity_cols()      

In [3]:
# get reactivity_error_* columns
reactivity_error_lf = lf.select(pl.col(reactivity_error_cols))
reactivity_error_df = reactivity_error_lf.collect(streaming=True)

reactivity_error_np = reactivity_error_df.to_numpy()
original_shape_err = reactivity_error_np.shape

# non-zero
reactivity_error_none_zero_np = reactivity_error_np[reactivity_error_np != 0]
non_zero_shape = reactivity_error_none_zero_np.shape

del reactivity_error_lf
del reactivity_error_df

percent_of_zeros(original_shape_err[0]*original_shape_err[1], non_zero_shape[0])

46.4259194854265%


In [3]:
# get reactivity_* columns
reactivity_lf = lf.select(pl.col(reactivity_cols))
reactivity_df = reactivity_lf.collect(streaming=True)

reactivity_np = reactivity_df.to_numpy()
original_shape = reactivity_np.shape

# non-zero
reactivity_non_zero_np = reactivity_np[reactivity_np != 0]
non_zero_shape = reactivity_non_zero_np.shape

del reactivity_lf
del reactivity_df

percent_of_zeros(original_shape[0]*original_shape[1], non_zero_shape[0])

23.56923878599666%


In [ ]:
reactivity_error_np

In [ ]:
# Ref : https://stackoverflow.com/questions/63991322/median-absolute-deviation-from-numpy-ndarray
def mad(data, axis=None, scale=1.4826, keepdims=False):
    """Median absolute deviation (MAD).
    
    Defined as the median absolute deviation from the median of the data. A
    robust alternative to stddev. Results should be identical to
    scipy.stats.median_absolute_deviation(), which does not take a keepdims
    argument.

    Parameters
    ----------
    data : array_like
        The data.
    scale : float, optional
        Scaling of the result. By default, it is scaled to give a consistent
        estimate of the standard deviation of values from a normal
        distribution.
    axis : numpy axis spec, optional
        Axis or axes along which to compute MAD.
    keepdims : bool, optional
        If this is set to True, the axes which are reduced are left in the
        result as dimensions with size one.

    Returns
    -------
    ndarray
        The MAD.
    """
    # keep dims here so that broadcasting works
    med = np.median(data, axis=axis, keepdims=keepdims)
    abs_devs = np.abs(data - med)
    return scale * np.median(abs_devs, axis=axis, keepdims=keepdims)

In [ ]:
# Mean absolute deviation
print(stats.median_abs_deviation(reactivity_error_np,axis=None))
print(mad(reactivity_error_np))

In [ ]:
# Coefficient of variation
stats.variation(reactivity_error_np, ddof=1,axis=None)